# CBM in C3M famework wit Joint Strategy

---
Goal of the notebook: .

Inputs of the notebook:
- 

Output of the notebook:
- prediction + interpretation layer for text classification.


Takeaways: 
- .
- .

In [ ]:
# !pip install -r requirements.txt

In [ ]:
import os
import sys
sys.path.append('../../run_experiments/')
sys.path.append('../../run_experiments/scripts')
sys.path.append('../../run_experiments/models')
sys.path.append('../../run_experiments/data')

import torch
import torch.nn as nn
from datasets import load_dataset
from sklearn.metrics import f1_score
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, Dataset, Subset


# model for CBM
# import fonction for getting PLM and tokenizer
from models.utils import load_model_and_tokenizer

# import config 
from config_agnews import Config

import json

# Library for Gemma
import re
from transformers import AutoTokenizer, AutoModelForCausalLM
from ast import literal_eval
from concepts_discovery_utils import extract_target_words, create_context_window, load_model, run_concepts_discovery



**Delete awkward warning**

In [ ]:
import logging
loggers = [logging.getLogger(name) for name in logging.root.manager.loggerDict]
for logger in loggers:
    if "transformers" in logger.name.lower():
        logger.setLevel(logging.ERROR)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# 0. autoreload

In [ ]:
#code for autoreload script associated with jupyter notebook
%load_ext autoreload
%autoreload 2

# 1.SETUP	 ENVIRONMENT VARIABLES

In [ ]:
# import config
from load_config import load_config

model_name = 'bert-base-uncased'    # 'bert-base-uncased' ou 'deberta-large' or 'gemma'
dataset    = 'agnews'               # 'movies' / 'agnews' / 'dbpedia' / 'medical'/ 'ledgar'/ n24news
annotation = 'combined_annotation'       # 'C3M' ou 'our_annotation' ou 'combined_annotation'
config = load_config(model_name, dataset)
config.annotation = annotation

# 2. Data Loading and Preparation


val dataloader


In [ ]:
# # import the data
from prepare_data import load_fc_prepare_data
prepare_data = load_fc_prepare_data(config.DATASET)
# train_loader, test_loader, val_loader, train_df, val_df, test_df = prepare_data(config)
_, _, val_loader, _, _, _ = prepare_data(config)

# 3. Launching experimentation

import concept list for C3M

In [ ]:
if config.annotation =='combined_annotation':
    cavs_type_arg = 'mean'
    with open(f"{config.SAVE_PATH}/blue_checkpoints/{config.model_name}/cavs/{cavs_type_arg}/sorted_macro_concepts_our_annotation.json", 'r') as f:
        data_our = json.load(f)
    with open(f"{config.SAVE_PATH}/blue_checkpoints/{config.model_name}/cavs/{cavs_type_arg}/sorted_macro_concepts_C3M.json", 'r') as f:
        data_c3m = json.load(f)
    # Tri de la liste par ordre décroissant selon la valeur (indice 1)
    data_triee_1 = sorted(data_our, key=lambda x: x[1], reverse=True)
    data_triee_2 = sorted(data_c3m, key=lambda x: x[1], reverse=True)

    from copy import copy
    starting_concepts = data_triee_1.copy()
    starting_concepts.extend(data_triee_2)
    
else:
    cavs_type_arg = 'mean'    
    with open(f"{config.SAVE_PATH}/blue_checkpoints/{config.model_name}/cavs/{cavs_type_arg}/sorted_macro_concepts_{config.annotation}.json", 'r') as f:
        data = json.load(f)
    starting_concepts = sorted(data, key=lambda x: x[1], reverse=True)

starting_concepts = [k for k,v in starting_concepts]
n_concept_start = len(starting_concepts)


### 1. Load the joint model

In [ ]:
if config.model_name == 'gemma':
    from models.jointCBMv2_gemma import JointModel
else :
    from models.jointCBMv2 import JointModel

In [ ]:
embedder_model, embedder_tokenizer, ModelXtoCtoY_layer, _ = load_model_and_tokenizer(config, n_concepts = n_concept_start)

embedder_model.to(config.device)
embedder_model.eval()

CBM_joint = JointModel(embedder_model, embedder_tokenizer, ModelXtoCtoY_layer, config, None, None)

if config.annotation in ['our_annotation', 'combined_annotation']:
    from concepts_bank_utils import clean_concept_name
    CBM_joint.concepts_name = [clean_concept_name(name) for name in starting_concepts]
else :
    CBM_joint.concepts_name = starting_concepts


## Launch xp

train and test augmented with C3M annotaion

In [ ]:
# # import the dataframe
from prepare_data import prepare_data_from_csv
df_aug_train, df_aug_test = prepare_data_from_csv(annotation  = config.annotation, config = config, return_test = True)

# # dataloaders
from concepts_bank_utils import create_dataloader
notre_loader_train = create_dataloader(df_aug_train, embedder_tokenizer, config.max_len, config.batch_size)
notre_loader_test = create_dataloader(df_aug_test, embedder_tokenizer, config.max_len, config.batch_size)

In [ ]:
CBM_joint.train_model(notre_loader_train, val_loader)                    
train_metrics = CBM_joint.evaluate_model(notre_loader_train, 'train', metrics_on_concepts = True)
val_metrics = CBM_joint.evaluate_model(val_loader, 'val', metrics_on_concepts = False)
test_metrics = CBM_joint.evaluate_model(notre_loader_test, 'test', metrics_on_concepts = True)

# Sauvegarder les perfs train et test des concept et aller dans le fichier new_heuristique pour voir
with open(f"{self.config.SAVE_PATH}blue_checkpoints/{self.config.model_name}/metrics_to_compare_agnews_{config.model_name}.json", 'w') as f:
    json.dump(test_metrics, f)